# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,-26.52,100,95,0.95,RU,1730761690
1,1,wynyard,-41.0000,145.7167,15.23,75,9,0.45,AU,1730761691
2,2,hadibu,12.6500,54.0333,25.76,79,12,2.44,YE,1730761692
3,3,keflavik,64.0049,-22.5624,10.24,93,100,11.83,IS,1730761693
4,4,port blair,11.6667,92.7500,26.90,78,99,2.45,IN,1730761695


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    hover_cols=['City'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Max Temp'] <= 30) & (city_data_df['Humidity'] <= 30) & (city_data_df['Cloudiness'] <= 50)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()
    
# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,huatabampo,26.8333,-109.6333,26.65,24,41,5.90,MX,1730761479
116,116,alaghsas,17.0187,8.0168,23.62,20,0,4.85,NE,1730761830
121,121,east porterville,36.0574,-118.9757,21.90,30,0,1.54,US,1730761836
137,137,uyuni,-20.4597,-66.8250,15.05,7,0,0.94,BO,1730761857
171,171,challapata,-18.9000,-66.7667,17.64,19,15,2.01,BO,1730761898


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,huatabampo,MX,26.8333,-109.6333,24,
116,alaghsas,NE,17.0187,8.0168,20,
121,east porterville,US,36.0574,-118.9757,30,
137,uyuni,BO,-20.4597,-66.8250,7,
171,challapata,BO,-18.9000,-66.7667,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000

params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
    'filter' : '',
    'bias': '',
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:' + ','.join([str(row['Lng']),str(row['Lat']),str(10000)])
    params["bias"] = 'proximity:' + ','.join([str(row['Lng']),str(row['Lat'])])

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
huatabampo - nearest hotel: No hotel found
alaghsas - nearest hotel: Hôtel Zagado
east porterville - nearest hotel: Best Western Porterville Inn
uyuni - nearest hotel: Hostel Jerian
challapata - nearest hotel: Residencial Marian
tonota - nearest hotel: No hotel found
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
nguigmi - nearest hotel: Guest PAM
el bauga - nearest hotel: No hotel found
dosso - nearest hotel: Galaxy Hotel
newman - nearest hotel: No hotel found
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
moapa valley - nearest hotel: No hotel found
santa maria del oro - nearest hotel: No hotel found
alipur - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
broken hill - nearest hotel: Ibis Styles
el matama - nearest hotel: الكوثر
victorica - nearest hotel: Hotel Mazamet
shujaabad - nearest hotel: No hotel found
patterson - nearest hotel: Hampton Inn & Suites
vallenar 

,City,Country,Lat,Lng,Humidity,Hotel Name
19,huatabampo,MX,26.8333,-109.6333,24,No hotel found
116,alaghsas,NE,17.0187,8.0168,20,Hôtel Zagado
121,east porterville,US,36.0574,-118.9757,30,Best Western Porterville Inn
137,uyuni,BO,-20.4597,-66.8250,7,Hostel Jerian
171,challapata,BO,-18.9000,-66.7667,19,Residencial Marian


## Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display
# Removing cities without hotels within 10,000 meter radius
hotel_df = hotel_df[(hotel_df['Hotel Name'] != 'No hotel found')]

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    hover_cols=['City','Hotel Name','Country'])

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)